# 158.739 Assignment 3
#### By Christina Xie 20008709

## 1. Schema Definition

I create a DB schema document that includes six tables that persist different attributes of variables in an Auckland real estate dataset. These tables cover property locations, property price, number of rooms, land size, property type, and geometry points. 

Before creating the DB schema, I read the real estate dataset first and use the index column as the unique identifier of each property. This is because the original dataset does not have a unique index for each address. What seems to be unique is not unique such as address and geometry point. This is the case for apartments, townhouses, and units. The index column is included in other tables and treated as the foreign key. 

The index is set as the primary key. The foreign key constraint checking system is enabled to study the relationships between tables. The property sale price is also included in all the tables to analyse the relationship between the price and different factors. 

The final DB schema document is shown as follows.
<img src=../figures/DB-schema.jpg width=300>

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3

from pylab import rcParams
rcParams['figure.dpi'] = 400
rcParams['lines.linewidth'] = 2 

sns.set_context("notebook", font_scale=1.2)
sns.set_style("darkgrid")

%matplotlib inline

# load ipython-sql library
%load_ext sql

In [65]:
# use the index as the unique identification of each property
listings = pd.read_csv('../datasets/akl_property_listings.csv')
listings.index.name='idx'
listings = listings.reset_index()

In [66]:
listings.head()

,idx,Location,Suburb,Area,Bedroom,Bathroom,LandSize_m2,PropertyType,SalePrice,Latitude,Longitude,geometry
0,0,"17/104 Rosier Road, Glen Eden",Glen Eden,West,2,1,NaN,Townhouse,819000,-36.91808,174.63994,POINT (174.63994 -36.91808)
1,1,"35 Hobson Street, Auckland Central",Auckland Central,Central,1,1,NaN,Apartment,549000,-36.84668,174.76202,POINT (174.76202 -36.84668)
2,2,"158 Onehunga Mall, Onehunga",Onehunga,Central,2,1,88.0,Apartment,760000,-36.92358,174.78530,POINT (174.7853 -36.92358)
3,3,"8 Ronayne Street, Auckland Central",Auckland Central,Central,3,2,NaN,Apartment,209000,-36.84914,174.77488,POINT (174.77488 -36.84914)
4,4,"430 Queen Street, Auckland Central",Auckland Central,Central,2,1,NaN,Apartment,409000,-36.85432,174.76324,POINT (174.76324 -36.85432)


In [5]:
# connect to the database
%sql sqlite:///../datasets/AucklandProperties.db

In [6]:
# enable Foreign Key constraint checking
%sql sqlite:///../datasets/AucklandProperties.db PRAGMA foreign_keys = ON;

Done.


[]

### Location table

In [72]:
%%sql sqlite:///../datasets/AucklandProperties.db
DROP TABLE IF EXISTS LOCATION;
CREATE TABLE LOCATION(  
    idx INTEGER NOT NULL UNIQUE,
    Location TEXT NOT NULL,
    Suburb TEXT NOT NULL,
    Area TEXT NOT NULL,
    SalePrice INTEGER NOT NULL,
    PRIMARY KEY (idx)
);

Done.
Done.


[]

### Price table

In [73]:
%%sql sqlite:///../datasets/AucklandProperties.db
DROP TABLE IF EXISTS PRICE;
CREATE TABLE PRICE(
    idx INTEGER NOT NULL UNIQUE,
    Location TEXT NOT NULL,
    SalePrice INTEGER NOT NULL,
    FOREIGN KEY (idx) REFERENCES LOCATION (idx)
);

Done.
Done.


[]

### Rooms table

In [74]:
%%sql sqlite:///../datasets/AucklandProperties.db
DROP TABLE IF EXISTS ROOMS;
CREATE TABLE ROOMS(
    idx INTEGER NOT NULL UNIQUE,
    Location TEXT NOT NULL,
    Bedroom INTEGER NOT NULL,
    Bathroom INTEGER NOT NULL,
    SalePrice INTEGER NOT NULL,
    FOREIGN KEY (idx) REFERENCES LOCATION (idx)
);

Done.
Done.


[]

### Land size table

In [85]:
%%sql sqlite:///../datasets/AucklandProperties.db
DROP TABLE IF EXISTS LAND;
CREATE TABLE LAND(
    idx INTEGER NOT NULL UNIQUE,
    Location TEXT NOT NULL,
    LandSize_m2 REAL,
    SalePrice INTEGER NOT NULL,
    FOREIGN KEY (idx) REFERENCES LOCATION (idx)
);

Done.
Done.


[]

### Property type table

In [76]:
%%sql sqlite:///../datasets/AucklandProperties.db
DROP TABLE IF EXISTS TYPES;
CREATE TABLE TYPES(
    idx INTEGER NOT NULL UNIQUE,
    Location TEXT NOT NULL,
    PropertyType TEXT NOT NULL,
    SalePrice INTEGER NOT NULL,
    FOREIGN KEY (idx) REFERENCES LOCATION (idx)
);

Done.
Done.


[]

### Geometry table

In [77]:
%%sql sqlite:///../datasets/AucklandProperties.db
DROP TABLE IF EXISTS GEOMETRY;
CREATE TABLE GEOMETRY(
    idx INTEGER NOT NULL UNIQUE,
    Location TEXT NOT NULL,
    Latitude REAL NOT NULL,
    Longitude REAL NOT NULL,
    geometry TEXT NOT NULL,
    SalePrice INTEGER NOT NULL,
    FOREIGN KEY (idx) REFERENCES LOCATION (idx)
);

Done.
Done.


[]

In [78]:
%sql sqlite:///../datasets/AucklandProperties.db PRAGMA foreign_keys

Done.


foreign_keys
1


## 2. DB Population

The next step is to populate the DB tables with the real estate data. I extract the relevant columns and create a pandas dataframe for each corresponding DB table. I also set the index as a newly created "idx" in each dataframe. Then each dataframe is read into SQLite tables. The persistence of data is checked after each reading.

In [46]:
listings.head()

,Location,Suburb,Area,Bedroom,Bathroom,LandSize_m2,PropertyType,SalePrice,Latitude,Longitude,geometry
idx,,,,,,,,,,,
0,"17/104 Rosier Road, Glen Eden",Glen Eden,West,2,1,NaN,Townhouse,819000,-36.91808,174.63994,POINT (174.63994 -36.91808)
1,"35 Hobson Street, Auckland Central",Auckland Central,Central,1,1,NaN,Apartment,549000,-36.84668,174.76202,POINT (174.76202 -36.84668)
2,"158 Onehunga Mall, Onehunga",Onehunga,Central,2,1,88.0,Apartment,760000,-36.92358,174.78530,POINT (174.7853 -36.92358)
3,"8 Ronayne Street, Auckland Central",Auckland Central,Central,3,2,NaN,Apartment,209000,-36.84914,174.77488,POINT (174.77488 -36.84914)
4,"430 Queen Street, Auckland Central",Auckland Central,Central,2,1,NaN,Apartment,409000,-36.85432,174.76324,POINT (174.76324 -36.85432)


In [80]:
location = listings[['idx','Location','Suburb','Area','SalePrice']].set_index('idx').copy()
price = listings[['idx','Location','SalePrice']].set_index('idx').copy()
rooms = listings[['idx','Location','Bedroom','Bathroom','SalePrice']].set_index('idx').copy()
land = listings[['idx','Location','LandSize_m2','SalePrice']].set_index('idx').copy()
types = listings[['idx','Location','PropertyType','SalePrice']].set_index('idx').copy()
geometry = listings[['idx','Location','Latitude','Longitude','geometry','SalePrice']].set_index('idx').copy()

In [79]:
connection = sqlite3.connect('../datasets/AucklandProperties.db')
location.to_sql('LOCATION', connection, if_exists='append')
connection.commit()

In [90]:
%%sql sqlres << 
SELECT * 
FROM LOCATION

 * sqlite:///../datasets/AucklandProperties.db
Done.
Returning data to local variable sqlres


In [82]:
connection = sqlite3.connect('../datasets/AucklandProperties.db')
price.to_sql('PRICE', connection, if_exists='append')
connection.commit()

In [91]:
%%sql sqlres << 
SELECT * 
FROM PRICE

 * sqlite:///../datasets/AucklandProperties.db
Done.
Returning data to local variable sqlres


In [83]:
connection = sqlite3.connect('../datasets/AucklandProperties.db')
rooms.to_sql('ROOMS', connection, if_exists='append')
connection.commit()

In [92]:
%%sql sqlres << 
SELECT * 
FROM ROOMS

 * sqlite:///../datasets/AucklandProperties.db
Done.
Returning data to local variable sqlres


In [86]:
connection = sqlite3.connect('../datasets/AucklandProperties.db')
land.to_sql('LAND', connection, if_exists='append')
connection.commit()

In [93]:
%%sql sqlres << 
SELECT * 
FROM LAND

 * sqlite:///../datasets/AucklandProperties.db
Done.
Returning data to local variable sqlres


In [87]:
connection = sqlite3.connect('../datasets/AucklandProperties.db')
types.to_sql('TYPES', connection, if_exists='append')
connection.commit()

In [94]:
%%sql sqlres << 
SELECT * 
FROM TYPES

 * sqlite:///../datasets/AucklandProperties.db
Done.
Returning data to local variable sqlres


In [88]:
connection = sqlite3.connect('../datasets/AucklandProperties.db')
geometry.to_sql('GEOMETRY', connection, if_exists='append')
connection.commit()

In [95]:
%%sql sqlres << 
SELECT * 
FROM GEOMETRY

 * sqlite:///../datasets/AucklandProperties.db
Done.
Returning data to local variable sqlres


## 3. SQLite Queries

### Property location and sale price

As shown in the table below, properties in North Shore city tend to have the highest average sale price, whereas those in Southern Auckland have the lowest average sale price.

In [120]:
%%sql sqlite://
SELECT 
    Area,
    ROUND(AVG(SalePrice),1) as AvgPrice
FROM LOCATION
GROUP BY Area
ORDER BY AvgPrice DESC

Done.


Area,AvgPrice
NorthShore,1417764.7
Other,1227920.4
East,1164873.8
Central,1160344.2
West,1085495.1
South,909244.6


The top 10 properties by sale price are shown as follows. Due to the website's mistake, some records are duplicate even though ```DISTINCT``` has been specified. However, the table still suggests that other than properties on islands, properties on Remuera, Auckland Central, Laingholm, and Epsom have a higher sale price. 

In [138]:
%%sql sqlite://
SELECT 
    DISTINCT Location,
    Suburb,
    SalePrice
FROM LOCATION
ORDER BY SalePrice DESC
LIMIT 10

Done.


Location,Suburb,SalePrice
"Pakatoa Island, Auckland Central",Auckland Central,50000000
"1 Pakatoa Island, Auckland Central",Auckland Central,50000000
"Pakatoa Island, Hauraki Gulf, Hauraki",Hauraki,50000000
"1 Pakatoa Island, Other Islands",Other Islands,50000000
". Pakatoa Island, Auckland Central",Auckland Central,40000000
"306 Cowes Bay Road, Waiheke Island",Waiheke Island,28000000
"218 Victoria Avenue, Remuera",Remuera,11295000
"51 Albert Street, Auckland Central",Auckland Central,11000000
"104 Kauri Point Road, Laingholm",Laingholm,8975000
"71 Watling Street, Epsom",Epsom,8050000


However, when we look at the bottom 10 properties by sale price, most of them are located in Auckland Central and Remuera. This may suggest that there could be other factors affecting the sale price. 

In [118]:
%%sql sqlite://
SELECT 
    DISTINCT Location,
    Suburb,
    SalePrice
FROM LOCATION
ORDER BY SalePrice ASC
LIMIT 10

Done.


Location,Suburb,SalePrice
"414/133 Beach Road, Auckland Central",Auckland Central,55000
"20/267 Remuera Road, Remuera",Remuera,69900
"1117/2 Beach Road, Auckland Central",Auckland Central,79000
"4/267 Remuera Road, Remuera",Remuera,85000
"7H/15 City Road, Auckland Central",Auckland Central,89000
"23-31 Hobson Street, Auckland Central",Auckland Central,89000
"5B/15 City Road, Auckland Central",Auckland Central,89000
"9I/15 City Road, Auckland Central",Auckland Central,89000
"Carpark 1/152 Hobson Street, Auckland Central",Auckland Central,96500
"40/139 Quay Street, Auckland Central",Auckland Central,99000


### Property type and sale price

Since there is not much variation on the location and suburb concerning the sale price, we now include their property types to analyse if different property types affect the sale price. It is done by joining the LOCATION and TYPES tables together.

As shown in the table of the top 10 properties by sale price, properties belong to either lifestyle property or house, except the one on Albert Street. In comparison, the bottom 10 table indicates that apartments and units tend to have a lower sale price.

In [135]:
%%sql sqlite://
SELECT 
    DISTINCT lo.Location, 
    lo.Suburb, 
    lo.SalePrice,
    ty.PropertyType
FROM TYPES as ty
LEFT JOIN LOCATION as lo
on ty.idx=lo.idx
ORDER BY lo.SalePrice DESC
LIMIT 10

Done.


Location,Suburb,SalePrice,PropertyType
"Pakatoa Island, Auckland Central",Auckland Central,50000000,Lifestyle Property
"1 Pakatoa Island, Auckland Central",Auckland Central,50000000,House
"Pakatoa Island, Hauraki Gulf, Hauraki",Hauraki,50000000,Lifestyle Property
"1 Pakatoa Island, Other Islands",Other Islands,50000000,Lifestyle Property
". Pakatoa Island, Auckland Central",Auckland Central,40000000,Lifestyle Property
"306 Cowes Bay Road, Waiheke Island",Waiheke Island,28000000,House
"306 Cowes Bay Road, Waiheke Island",Waiheke Island,28000000,Lifestyle Property
"218 Victoria Avenue, Remuera",Remuera,11295000,House
"51 Albert Street, Auckland Central",Auckland Central,11000000,Apartment
"104 Kauri Point Road, Laingholm",Laingholm,8975000,House


In [139]:
%%sql sqlite://
SELECT 
    DISTINCT lo.Location, 
    lo.Suburb, 
    lo.SalePrice,
    ty.PropertyType
FROM TYPES as ty
LEFT JOIN LOCATION as lo
on ty.idx=lo.idx
ORDER BY lo.SalePrice ASC
LIMIT 10

Done.


Location,Suburb,SalePrice,PropertyType
"414/133 Beach Road, Auckland Central",Auckland Central,55000,Apartment
"20/267 Remuera Road, Remuera",Remuera,69900,Apartment
"1117/2 Beach Road, Auckland Central",Auckland Central,79000,Apartment
"4/267 Remuera Road, Remuera",Remuera,85000,Apartment
"7H/15 City Road, Auckland Central",Auckland Central,89000,Apartment
"23-31 Hobson Street, Auckland Central",Auckland Central,89000,Unit
"5B/15 City Road, Auckland Central",Auckland Central,89000,Apartment
"9I/15 City Road, Auckland Central",Auckland Central,89000,Apartment
"Carpark 1/152 Hobson Street, Auckland Central",Auckland Central,96500,Carpark
"40/139 Quay Street, Auckland Central",Auckland Central,99000,Apartment


### Land size and sale price

I am also interested to see that if properties having higher sale prices also have larger land sizes. I join the LOCATION and LAND tables to find out if similar addresses are displayed. 

The first table shows the top 10 properties by land size. As seen from the results, a larger land size is not necessarily related to a higher sale price.  

In [148]:
%%sql sqlite://
SELECT * 
FROM LAND
WHERE LandSize_m2 IS NOT NULL
ORDER BY LandSize_m2 DESC
LIMIT 10

Done.


idx,Location,LandSize_m2,SalePrice
2257,"948 Whangaripo Valley Road, Wellsford",1802000.0,1600000
2367,"438 Komokoriki Hill Road, Makarau",925000.0,2300000
77,"4193 Kaipara Coast Highway, Tauhoa",529000.0,2500000
2730,"169B Bayer Road, Waiwera",502000.0,625000
1882,"475 Peak Road, Helensville",463000.0,2950000
1445,"210 Staniforth Road, Wellsford",447000.0,3150000
1045,"Lot 1 Brott Road, Kaipara Flats",397000.0,849000
2731,"68 Urquhart Road, Karaka",387000.0,7500000
2079,"Lot 2 Rodney Road, Leigh",359000.0,4500000
2727,"Lot 2/Lot 2 Rodney Road, Leigh",359000.0,4500000


The second table displays the top 10 properties by the sale price. Similar results are found. Although properties in Laingholm, Epsom, and Manukau Heights have smaller land sizes, their sale prices are high.

In [145]:
%%sql sqlite://
SELECT 
    DISTINCT lo.Location, 
    lo.Suburb, 
    lo.SalePrice,
    la.LandSize_m2
FROM LAND as la
LEFT JOIN LOCATION as lo
on la.idx=lo.idx
WHERE la.LandSize_m2 IS NOT NULL
ORDER BY lo.SalePrice DESC
LIMIT 10

Done.


Location,Suburb,SalePrice,LandSize_m2
"Pakatoa Island, Auckland Central",Auckland Central,50000000,241000.0
"1 Pakatoa Island, Auckland Central",Auckland Central,50000000,241000.0
"Pakatoa Island, Hauraki Gulf, Hauraki",Hauraki,50000000,241000.0
"1 Pakatoa Island, Other Islands",Other Islands,50000000,241000.0
". Pakatoa Island, Auckland Central",Auckland Central,40000000,241000.0
"306 Cowes Bay Road, Waiheke Island",Waiheke Island,28000000,315000.0
"104 Kauri Point Road, Laingholm",Laingholm,8975000,7234.0
"71 Watling Street, Epsom",Epsom,8050000,2258.0
"481 Redoubt Road, Manukau Heights",Manukau Heights,7500000,17000.0
"68 Urquhart Road, Karaka",Karaka,7500000,387000.0


## 4. DB Views

### View 1: Median comparison

The latest report from Real Estate Institute indicates that the Auckland median price has reached $\$$1.12 million in April. I will calculate the number of properties whose sale price is above or below the median price. In this dataset, about one-third of the properties have sale prices above the Auckland median price with an average sale price of $\$$2,286,360.5.

In [158]:
%%sql sqlite://
CREATE VIEW MEDIAN_COMPARISON AS
    SELECT 
        MedianComparison,
        Count(MedianComparison) as Count,
        ROUND(AVG(SalePrice),1) as AvgPrice
    FROM
        (SELECT 
            SalePrice, 
            Location,
            CASE WHEN SalePrice > 1120000 THEN 'Above median'
            ELSE 'Below median' END
            AS MedianComparison
        FROM LOCATION)
    GROUP BY MedianComparison
    ORDER BY Count ASC

 * sqlite:///../datasets/AucklandProperties.db
Done.


[]

In [159]:
%%sql sqlite://
SELECT *
FROM MEDIAN_COMPARISON

Done.


MedianComparison,Count,AvgPrice
Above median,883,2286360.5
Below median,1869,654553.5


### View 2: North Shore property types

The second view shows the number of different property types in North Shore city. The dominant types are houses and apartments.

In [175]:
%%sql sqlite://
CREATE VIEW TYPE_COUNT_NORTHSHORE AS
    SELECT PropertyType, COUNT(PropertyType) as Count_NorthShore
    FROM (SELECT
            lo.Area,
            lo.SalePrice,
            ty.PropertyType
        FROM TYPES as ty
        LEFT JOIN LOCATION as lo
        on ty.idx=lo.idx)
    WHERE Area = 'NorthShore'
    GROUP BY PropertyType
    ORDER BY Count_NorthShore DESC

Done.


[]

In [176]:
%%sql sqlite://
SELECT *
FROM TYPE_COUNT_NORTHSHORE

Done.


PropertyType,Count_NorthShore
House,133
Apartment,121
Townhouse,58
Section,21
Unit,12
Home & Income,5
Lifestyle Section,4
Lifestyle Property,3
